In [7]:
import sys
sys.path.append("scripts")
from importPy import *
import importPy

D:\Dev\ProjetCNNAPICE\Python\scripts/
__init__.py
detect.py
eega_maskmatrix.py
eega_summaryartifacts.py
eega_tArtifacts.py
eega_tDefBTBC.py
eega_tIncShortBad.py
eega_tMask.py
eega_tRejAmp.py
eega_tRejAmpElecVar.py
eega_tRejCorrCh.py
eega_tRejFastChange.py
eega_tRejPwr.py
eega_tRejRunningAvg.py
eega_tRejShortGood.py
eega_tRejTimeVar.py
eega_ZscoreForArt.py
eeg_checkart.py
__init__.py
eega_tInterpSpatialEEG.py
eega_tInterpSpatialSegmentEEG.py
eega_tTargetPCAxElEEG.py
__init__.py
eega_plot_artifacts.py
eega_chanedit.py
eega_filt.py
eega_getoptions.py
eega_RunAll.py
eega_str2func.py
matlab_functions.py
__init__.py
eega_latencyevent.py
__init__.py
eega_importdata.py
eega_importinfoevents.py
eega_removeevent.py
__init__.py
eega_demean.py
eega_epoch.py
eega_refavg.py
eega_rmvbaddata.py
__init__.py
__init__.py
example_APICE_ArtPP_BadEl.py
example_APICE_ArtPP_Jump.py
example_APICE_ArtPP_Mot1.py
example_APICE_ArtPP_Mot2.py
example_APICE_Interpolation.py
__init__.py


In [14]:

# Current path:
Path0 = 'D:/Dev/ProjetCNNAPICE/Matlab/eeg_preprocessing_main/examples';
# Folder where the scripts defining the parameters are:
Path2Parameters = os.path.join(Path0,'parameters');
# Channels location file
filechanloc = os.path.join(Path0,'DATA','ElectrodesLayout','GSN-HydroCel-128.sfp');
# Folder where the event files are:
Path2DataEvent = os.path.join(Path0,'DATA','evt');
# Folder where the data imported to EEGLAB is: 
Path2DataSet = os.path.join(Path0,'DATA','set');
# Folder where the continuos preprocess data will be saved
Path2DataPrp = os.path.join(Path0,'DATA','prp');
# Folder where the ERPs will be saved
Path2DataERP = os.path.join(Path0,'DATA','erp');




## ========================================================================
## PARAMETERS
# Define the parameters

Ppp = ParametersPreProcessing()
# -------------------------------------------------------------------------
# Parameters for importing the data
# -------------------------------------------------------------------------

# Name of the files to read from the Path2DataSet folder
Ppp.Files2Read = '*.set' 

# Add extra information in an even file (.evt) located in Path2DataEventimport 
#                         .
# To remove this step, delete the line with eega_importinfoevents!!!
#                         .
# This step should be avoided unless extra information in the event 
# file is not imported from the raw data to the EEGLAB structure. 
# The function adding events has been created to add events sent by 
# Psychtoolbox to EGI. The addition of new events will not work with other
# acquisition systems. Customized functions might be required

# Event to use to align differences in time offsets between the event file 
# and the EEG structure
Ppp.importevnt0 = 'STRT'    
	

# -------------------------------------------------------------------------
# Parameters for correcting the events
# -------------------------------------------------------------------------

# Correct the latency of a given event using Digital Input events (DINs)
#                         .
# To remove this step, delete the line with eega_latencyevent!!!
#                         .
# The latency of the events 'Icue' 'Iout' 'Ieye' are corrected by the latency of 'DIN6'

Ppp.eventcorrlat=EventCorrlat()
Ppp.eventcorrlat.event1 = ['Icue', 'Iout', 'Ieye']
Ppp.eventcorrlat.event2 = 'DIN6'    
# Delete unuseful events
#                         .
# To remove this step, delete the line with eega_removeevent!!!
#                         .
# Events to delete
Ppp.eventrmv = ['DIN6'] 


# -------------------------------------------------------------------------
# Parameters for filtering
# -------------------------------------------------------------------------

# High pass filter
Ppp.filt_highpass   = 0.1 

# Low pass filter
Ppp.filt_lowpass    = 40 


# -------------------------------------------------------------------------
# Parameters for artifacts detection
# -------------------------------------------------------------------------

# Generate the structures with the parameters for the artefact detection
# The functions to generate the structures should be in the path (Path2Parameters)
# These functions should be modified to change the algorithms applied for
# artifacts detection

# In this example, we use a relative threshold of 3 for all types of
# artifacts (APICE(3))

# Algorithms to detect bad electrodes
Ppp.ArtBadEl = example_APICE_ArtPP_BadEl(3) 
# Algorithms to detect jumps in the signal
Ppp.ArtJump = example_APICE_ArtPP_Jump(3) 
# Algorithms to detect motion artifacts
Ppp.ArtMot1 = example_APICE_ArtPP_Mot1(3) 
# Algorithms to detect motion artifacts
Ppp.ArtMot2 = example_APICE_ArtPP_Mot2(3) 


# -------------------------------------------------------------------------
# Parameters to define Bad Times (BT) and Bad Channels (BC) 
# -------------------------------------------------------------------------

# Limits for the proportion of BT to define a BC (the last value is the final/effective one)
Ppp.BCall.nbt           = [0.70, 0.50, 0.30] 
# Limits for the proportion of BC to define a BT (the last value is the final/effective one)
Ppp.BTall.nbc           = [0.70, 0.50, 0.30] 
# Shorter intervals between bad segments will be marked as bad
Ppp.BTall.minGoodTime   = 1.000 
# Shorter periods will not be considered as bad
Ppp.BTall.minBadTime    = 0.100    
# Also mark as bad surronding samples within this value 
Ppp.BTall.maskTime      = 0.500         

# Create a cell array with all the input to use later
inputsDefBTBC = [Ppp.BTall.nbc, Ppp.BCall.nbt, Ppp.BCall.nbt, 'keeppre', 0, 
    'minBadTime', Ppp.BTall.minBadTime, 'minGoodTime', Ppp.BTall.minGoodTime, 'maskTime', Ppp.BTall.maskTime] 


# -------------------------------------------------------------------------
# Parameters to generate the report
# -------------------------------------------------------------------------

# Pattern to find in the names to create the table (subjetcs identifier). 
# If empty, the whole files name are used
Ppp.report.patternname = 'data'   
# Number of caracters to use from the begiging of the patter to create the table. 
# If empty, the name is extracted from the begign of the patter till the
# end of the name
Ppp.report.patternn = [] 

## ========================================================================
## RUN THE SET OF FUNCTIONS FOR ALL THE FILES

# Name of the files to run the processes
FilesIn = '*.set' 

# String to add at the begign of the output filename
FilesOut = 'prp_' 

# Define how to run eega_RunAll
# - If 1, run to all the files in the input folder
# - If 2, run to the files in the input folder do not present in the ouput folder (files still not analyzed)
runto = 1 

# Import all the files in the input folder

eega_RunAll(FilesIn, 
            Path2DataSet, 
            Path2DataPrp, 
            ['eega_chanedit',               [filechanloc],
            'eega_importinfoevents',        [Path2DataEvent, Ppp.importevnt0], 
            'eega_latencyevent',            [Ppp.eventcorrlat.event2, Ppp.eventcorrlat.event1], 
            'eega_removeevent',             [Ppp.eventrmv,  [], 'all'], 
            'eega_demean',                  [], 
            'eega_filt',               [[], Ppp.filt_lowpass], 
            'eega_filt',               [Ppp.filt_highpass, 0], 
            'eega_tArtifacts',              [Ppp.ArtBadEl, 'KeepRejPre', 1], 
            'eega_tArtifacts',              [Ppp.ArtMot1, 'KeepRejPre', 1], 
            'eega_tArtifacts',              [Ppp.ArtJump, 'KeepRejPre', 1],             'eega_tDefBTBC',                inputsDefBTBC, 
            'prename',FilesOut,'runto', runto]) 



<function eega_chanedit at 0x000001E6C45A7700> ['D:/Dev/ProjetCNNAPICE/Matlab/eeg_preprocessing_main/examples\\DATA\\ElectrodesLayout\\GSN-HydroCel-128.sfp']
<function eega_importinfoevents at 0x000001E6C45AF670> ['D:/Dev/ProjetCNNAPICE/Matlab/eeg_preprocessing_main/examples\\DATA\\evt', 'STRT']
<function eega_latencyevent at 0x000001E6C45AF280> ['DIN6', ['Icue', 'Iout', 'Ieye']]
<function eega_removeevent at 0x000001E6C45AF700> [['DIN6'], [], 'all']
<function eega_demean at 0x000001E6C45AF940> []
<function eega_filt at 0x000001E6C45A7790> [[], 40]
<function eega_filt at 0x000001E6C45A7790> [0.1, 0]
<function eega_tArtifacts at 0x000001E6C4598040> [<mymodule.example_APICE_ArtPP_BadEl object at 0x000001E6C46863A0>, 'KeepRejPre', 1]
<function eega_tArtifacts at 0x000001E6C4598040> [<mymodule.example_APICE_ArtPP_Mot1 object at 0x000001E6C46861F0>, 'KeepRejPre', 1]
<function eega_tArtifacts at 0x000001E6C4598040> [<mymodule.example_APICE_ArtPP_Jump object at 0x000001E6C46863D0>, 'KeepRejPr

<function eega_filt at 0x000001E6C45A7790>
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 165 samples (0.330 sec)

<function eega_filt at 0x000001E6C45A7790>
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 16501 samples (33.002 sec)

<fun

NameError: name 'Cnf' is not defined